In [1]:
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings('ignore')
df_tweets_alldata = pd.read_csv('tweets.csv')
df_market_alldata = pd.read_csv('marketdata.csv')

In [2]:
# Summary data on a daily basis
from datetime import datetime
df_tweets = df_tweets_alldata[['timestamp', 'tickers', 'neg', 'pos', 'neu', 'compound']]
df_tweets['timestamp'] = pd.to_datetime(df_tweets['timestamp'], format='ISO8601')

# If tickers include BTC
for i in range(df_tweets.shape[0]):
    ticker = df_tweets.iloc[i, 1]
    if 'BTC' in ticker:
        df_tweets.iloc[i, 1] = 1
    else:
        df_tweets.iloc[i, 1] = 0

# Group the data
daily_data = df_tweets.groupby(df_tweets['timestamp'].dt.date)
tweets_summary = []
for date, group in daily_data:
    # How many times btc is mentioned
    btc_mentioned = group['tickers'].sum()
    # How many tweets per day
    data_count = len(group)
    # Average sentiment of BTC
    btc_neg_avg = group['neg'][group['tickers']==1].mean()
    btc_pos_avg = group['pos'][group['tickers']==1].mean()
    btc_neu_avg = group['neu'][group['tickers']==1].mean()
    btc_com_avg = group['compound'][group['tickers']==1].mean()
    # Average sentiment of all tweets
    neg_avg = group['neg'].mean()
    pos_avg = group['pos'].mean()
    neu_avg = group['neu'].mean()
    com_avg = group['compound'].mean()
    
    tweets_summary.append({
        'date': date,
        'BTCtweets': btc_mentioned,
        'Totaltweets': data_count,
        'btc_neg_avg': btc_neg_avg,
        'btc_pos_avg': btc_pos_avg,
        'btc_neu_avg': btc_neu_avg,
        'btc_com_avg': btc_com_avg,
        'neg_avg': neg_avg,
        'pos_avg': pos_avg,
        'neu_avg': neu_avg,
        'com_avg': com_avg
    })

tweets_summary = pd.DataFrame(tweets_summary)
tweets_summary = tweets_summary.set_index('date')
tweets_summary

,BTCtweets,Totaltweets,btc_neg_avg,btc_pos_avg,btc_neu_avg,btc_com_avg,neg_avg,pos_avg,neu_avg,com_avg
date,,,,,,,,,,
2021-12-27,3,28,0.000000,0.000000,1.000000,0.000000,0.035000,0.110107,0.854893,0.160389
2022-01-02,3,21,0.000000,0.173333,0.826667,0.360833,0.037714,0.100143,0.862190,0.149724
2022-01-03,32,192,0.074000,0.078000,0.847938,0.018784,0.031240,0.113922,0.854833,0.167254
2022-01-04,21,170,0.060048,0.065190,0.874762,-0.024548,0.045288,0.097582,0.857118,0.109683
2022-01-05,55,159,0.049582,0.089236,0.861182,0.068989,0.044516,0.115711,0.839780,0.126904
...,...,...,...,...,...,...,...,...,...,...
2023-11-11,19,95,0.043526,0.141211,0.815263,0.253489,0.029400,0.111147,0.859453,0.171907
2023-11-12,13,78,0.034769,0.135846,0.829385,0.318123,0.015897,0.125385,0.858718,0.294297
2023-11-13,18,56,0.032722,0.110944,0.856333,0.291211,0.043268,0.109250,0.847482,0.264696


In [3]:
# Drop useless columns and rename columns to use pandas_ta
market_data = df_market_alldata.drop('volumeto', axis=1)
market_data['time'] = pd.to_datetime(market_data['time'], format='ISO8601') 
market_data['time'] = market_data['time'].dt.date
market_data.set_index(['time'], inplace=True)
market_data.rename(columns={'volumefrom': 'volume'}, inplace=True)

# Use pandas_ta to calculate some tech factors
import pandas_ta as ta
market_data.ta.mom(close='close', append=True) # 10-day momentum
market_data.ta.sma(length=5, append=True) # 5-day sma
market_data.ta.sma(length=10, append=True) # 10-day sma
market_data.ta.rsi(append=True) # RSI
market_data.ta.mfi(append=True) # MFI

# Caculate next day price change as target variable
# If next day's close>open, nxt_day_return = 1, meaning a positive return
# If next day's close<open, nxt_day_return = 0, meaning a negative return
market_data['nxt_day_return'] = np.where(
    market_data['close'].shift(-1) > market_data['open'].shift(-1), 1, 0)
# Drop several dates with NaN and the last date which does not have nxt_day_return
market_data.dropna(axis=0, inplace=True)
market_data.drop(market_data.index[-1], inplace=True)

market_data

,high,low,open,volume,close,MOM_10,SMA_5,SMA_10,RSI_14,MFI_14,nxt_day_return
time,,,,,,,,,,,
2010-07-31,0.06889,0.056,0.06270,4022.25,0.06785,-0.01136,0.06387,0.060143,56.654482,67.387698,0
2010-08-01,0.06785,0.060,0.06785,2601.00,0.06110,0.00268,0.06409,0.060411,51.332223,62.158907,0
2010-08-02,0.06330,0.060,0.06110,3599.00,0.06110,-0.00152,0.06453,0.060259,51.332223,55.445202,0
2010-08-03,0.06500,0.059,0.06110,9821.46,0.06000,0.00546,0.06255,0.060805,50.436721,44.318115,0
2010-08-04,0.06231,0.057,0.06000,3494.00,0.05700,0.00650,0.06141,0.061455,47.978415,41.970684,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-03,66925.93000,64514.870,65466.81000,27792.87,65986.11000,-1222.22000,68416.43800,69209.242000,48.417310,48.993811,1
2024-04-04,69343.32000,65081.420,65986.11000,36673.50,68523.79000,-1368.47000,68195.19800,69072.395000,53.432027,48.908526,0
2024-04-05,68772.34000,65994.650,68523.79000,32904.80,67860.53000,-2129.69000,67504.72200,68859.426000,52.008874,49.225660,1


In [4]:
# Concat two df
df_concat = pd.concat([tweets_summary, market_data], axis=1, join='inner')
df_concat.to_csv('concat_data.csv', encoding='utf-8', index=True)